In [ ]:
!pip install unidecode python-Levenshtein
!pip install evaluate
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: p

# DOWNLOAD DATASET

In [ ]:
from glob import glob
import os
import numpy as np
import pandas as pd
import sys
import re

import nltk
nltk.download('punkt') # Download the Punkt tokenizer models if you haven't already


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!git clone https://github.com/duyvuleo/VNTC.git

Cloning into 'VNTC'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Receiving objects: 100% (39/39), 160.90 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (15/15), done.
Filtering content: 100% (2/2), 168.95 MiB | 51.29 MiB/s, done.


In [ ]:
!pip install patool


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
import patoolib
patoolib.extract_archive("/content/VNTC/Data/27Topics/Ver1.1/Train.rar", outdir="/content/Train")
patoolib.extract_archive("/content/VNTC/Data/27Topics/Ver1.1/Test.rar", outdir="/content/Test")


INFO patool: Extracting /content/VNTC/Data/27Topics/Ver1.1/Train.rar ...
INFO:patool:Extracting /content/VNTC/Data/27Topics/Ver1.1/Train.rar ...
INFO patool: ... creating output directory `/content/Train'.
INFO:patool:... creating output directory `/content/Train'.
INFO patool: running /usr/bin/unrar x -- /content/VNTC/Data/27Topics/Ver1.1/Train.rar
INFO:patool:running /usr/bin/unrar x -- /content/VNTC/Data/27Topics/Ver1.1/Train.rar
INFO patool:     with cwd='/content/Train', input=''
INFO:patool:    with cwd='/content/Train', input=''
INFO patool: ... /content/VNTC/Data/27Topics/Ver1.1/Train.rar extracted to `/content/Train'.
INFO:patool:... /content/VNTC/Data/27Topics/Ver1.1/Train.rar extracted to `/content/Train'.
INFO patool: Extracting /content/VNTC/Data/27Topics/Ver1.1/Test.rar ...
INFO:patool:Extracting /content/VNTC/Data/27Topics/Ver1.1/Test.rar ...
INFO patool: ... creating output directory `/content/Test'.
INFO:patool:... creating output directory `/content/Test'.
INFO patool

'/content/Test'

In [ ]:
file_paths_train = glob('/content/Train/new train/*/*.txt')
file_paths_test = glob('/content/Test/new test/*/*.txt')
file_paths = file_paths_train + file_paths_test
len(file_paths)

26451

# PREPARE DATA

In [ ]:
from tqdm.autonotebook import tqdm

<ipython-input-7-d05506e76d3e>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
data = []
count = 0
for path in tqdm(file_paths):
  with open(path,'rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16")
    contents = contents.replace("\r\n",". ")
    sentences = nltk.sent_tokenize(contents)

    data.extend(sentences)

# Shuffle
import random

random.shuffle(data)

  0%|          | 0/26451 [00:00<?, ?it/s]

In [ ]:
data_processed = []

# Use tqdm to create a progress bar
for content in tqdm(data):
    if len(content) > 50:
        content = content.replace('..', '.'). \
                            replace('"', ''). \
                            replace(': .', '.'). \
                            replace('. .', '.'). \
                            replace("'", ''). \
                            replace('“', ''). \
                            replace('”', '')

        # Check length of words
        words = content.split()
        for word in words:
            if len(word) > 10:
                content = None

        if content == None:
            continue

        if content[0] == '-':
            content = content.replace('-', '')

        if content[0] == ' ':
            content = content[1:]

        if '. - ' in content:
            content = content.replace('. - ', '. ')

        if '.. ' in content:
            content = content.replace('.. ', '... ')

        if '?.' in content:
            content = content.replace('?.', '?')

        if ' ?.' in content:
            content = content.replace(' ?.', '?')

        if '&' in content:
            continue

        if '( ' in content:
            content = content.replace('( ', '(')

        if ' )' in content:
            content = content.replace(' )', ')')

        if '()' in content:
            content = content.replace('()', '')

        if content[-2:] == ' .':
            content = content.replace(content[-2:], '.')
            #print(content)

        if '… ' in content:
            content = content.replace('… ', '... ')

        if '….' in content:
            content = content.replace('….', '...')

        if '…' in content:
            continue

        if ':.' in content:
            continue

        if content.isascii():
            continue

        if '  ' in content:
            content = content.replace('  ', ' ')

        match = re.search(r"\.\s\d+\.$", content)
        if match:
            content = re.sub(r"\.\s\d+\.$", ".", content)

        if content[0] == '·':
            continue


        data_processed.append(content)


  0%|          | 0/653815 [00:00<?, ?it/s]

In [ ]:
len(data_processed)

369673

In [ ]:
# extract Latin- characters only
alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!/\"\',\-\.:;?_\(\)]+$'
data=[]

for sentence in data_processed:
    if re.match(alphabet, sentence.lower()):
        sentence = sentence.strip()

        data.append(sentence)

print(len(data))

359993


# CHIA TRAIN VÀ TEST

In [ ]:
test_size = int(0.1*len(data))

train_data = data[:-test_size]
test_data = data[-test_size:]

# NGRAM TRAIN

In [ ]:
import nltk
import pandas as pd
from tqdm import tqdm
import re
import itertools
import sys
import numpy as np

def extract_phrases(text):
        return re.findall(r'\w[\w ]+', text)

def gen_ngrams(text, n=5):
    tokens = text.split()

    return [tokens] if len(tokens) < n else nltk.ngrams(text.split(), n)

def processing(data):

    phrases = itertools.chain.from_iterable(extract_phrases(text) for text in data)
    phrases = [p.strip() for p in phrases if len(p.split()) > 1]

    # gen ngrams
    list_ngrams = []
    for p in tqdm(phrases):

        list_ngrams.extend(
            " ".join(ngr)
            for ngr in gen_ngrams(p, 5)
            if len(" ".join(ngr)) < 46
        )

    return list_ngrams

train_ngrams = processing(train_data)


100%|██████████| 302492/302492 [00:03<00:00, 84939.76it/s] 


In [ ]:
len(train_ngrams)

2026887

# ADD NOISE

In [ ]:
import numpy as np
import re
import unidecode
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string


class SynthesizeData(object):
    """
    Uitils class to create artificial miss-spelled words
    Args:
        vocab_path: path to vocab file. Vocab file is expected to be a set of words, separate by ' ', no newline charactor.
    """

    def __init__(self, vocab_path=""):

        self.tokenizer = word_tokenize
        self.word_couples = [['sương', 'xương'], ['sĩ', 'sỹ'], ['sẽ', 'sẻ'], ['sã', 'sả'], ['sả', 'xả'], ['sẽ', 'sẻ'],
                             ['mùi', 'muồi'],
                             ['chỉnh', 'chỉn'], ['sữa', 'sửa'], ['chuẩn', 'chẩn'], ['lẻ', 'lẽ'], ['chẳng', 'chẵng'],
                             ['cổ', 'cỗ'],
                             ['sát', 'xát'], ['cập', 'cặp'], ['truyện', 'chuyện'], ['xá', 'sá'], ['giả', 'dả'],
                             ['đỡ', 'đở'],
                             ['giữ', 'dữ'], ['giã', 'dã'], ['xảo', 'sảo'], ['kiểm', 'kiễm'], ['cuộc', 'cục'],
                             ['dạng', 'dạn'],
                             ['tản', 'tảng'], ['ngành', 'nghành'], ['nghề', 'ngề'], ['nổ', 'nỗ'], ['rảnh', 'rãnh'],
                             ['sẵn', 'sẳn'],
                             ['sáng', 'xán'], ['xuất', 'suất'], ['suôn', 'suông'], ['sử', 'xử'], ['sắc', 'xắc'],
                             ['chữa', 'chửa'],
                             ['thắn', 'thắng'], ['dỡ', 'dở'], ['trải', 'trãi'], ['trao', 'trau'], ['trung', 'chung'],
                             ['thăm', 'tham'],
                             ['sét', 'xét'], ['dục', 'giục'], ['tả', 'tã'], ['sông', 'xông'], ['sáo', 'xáo'],
                             ['sang', 'xang'],['tôi','tui'],
                             ['ngã', 'ngả'], ['xuống', 'suống'], ['xuồng', 'suồng']]

        self.char_couples = [['i', 'y'], ['s', 'x'], ['gi', 'd'],
                             ['ă', 'â'], ['ch', 'tr'], ['ng', 'n'],['b','p'],['r','g'],
                             ['nh', 'n'], ['ngh', 'ng'], ['ục', 'uộc'], ['o', 'u'],
                             ['ă', 'a'], ['o', 'ô'], ['ả', 'ã'], ['ổ', 'ỗ'], ['ủ', 'ũ'], ['ễ', 'ể'],
                             ['e', 'ê'], ['à', 'ờ'], ['ằ', 'à'], ['ẩn', 'uẩn'], ['ẽ', 'ẻ'], ['ùi', 'uồi'], ['ă', 'â'],
                             ['ở', 'ỡ'], ['ỹ', 'ỷ'], ['ỉ', 'ĩ'], ['ị', 'ỵ'],
                             ['ấ', 'á'], ['n', 'l'], ['qu', 'w'], ['ph', 'f'], ['d', 'z'], ['c', 'k'], ['qu', 'q'],
                             ['i', 'j'], ['gi', 'j'],['gi','z'],['qu','v'],['m','n']
                             ]

        self.teencode_dict = {'mình': ['mk', 'mik', 'mjk'], 'vô': ['zô', 'zo', 'vo'], 'vậy': ['zậy', 'z', 'zay', 'za'],
                              'phải': ['fải', 'fai', ], 'biết': ['bit', 'biet'],
                              'rồi': ['rùi', 'ròi', 'r'], 'bây': ['bi', 'bay'], 'giờ': ['h', ],
                              'không': ['k', 'ko', 'khong', 'hk', 'hong', 'hông', '0', 'kg', 'kh', ],
                              'đi': ['di', 'dj', ], 'gì': ['j', ], 'em': ['e', ], 'được': ['dc', 'đc', ], 'tao': ['t'],
                              'tôi': ['t'], 'chồng': ['ck'], 'vợ': ['vk']

                              }

        self.typo={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
                  "ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
                  "ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
                  "ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
                  "ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
                  "Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
                  "Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
                  "Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
                  "Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
                  "Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}
        # self.typo = {"ă": ["aw", "a8"], "â": ["aa", "a6"], "á": ["as", "a1"], "à": ["af", "a2"], "ả": ["ar", "a3"],
        #              "ã": ["ax", "a4"], "ạ": ["aj", "a5"], "ắ": ["aws", "ă1"], "ổ": ["oor", "ô3"], "ỗ": ["oox", "ô4"],
        #              "ộ": ["ooj", "ô5"], "ơ": ["ow", "o7"],
        #              "ằ": ["awf", "ă2"], "ẳ": ["awr", "ă3"], "ẵ": ["awx", "ă4"], "ặ": ["awj", "ă5"], "ó": ["os", "o1"],
        #              "ò": ["of", "o2"], "ỏ": ["or", "o3"], "õ": ["ox", "o4"], "ọ": ["oj", "o5"], "ô": ["oo", "o6"],
        #              "ố": ["oos", "ô1"], "ồ": ["oof", "ô2"],
        #              "ớ": ["ows", "ơ1"], "ờ": ["owf", "ơ2"], "ở": ["owr", "ơ2"], "ỡ": ["owx", "ơ4"], "ợ": ["owj", "ơ5"],
        #              "é": ["es", "e1"], "è": ["ef", "e2"], "ẻ": ["er", "e3"], "ẽ": ["ex", "e4"], "ẹ": ["ej", "e5"],
        #              "ê": ["ee", "e6"], "ế": ["ees", "ê1"], "ề": ["eef", "ê2"],
        #              "ể": ["eer", "ê3"], "ễ": ["eex", "ê3"], "ệ": ["eej", "ê5"], "ú": ["us", "u1"], "ù": ["uf", "u2"],
        #              "ủ": ["ur", "u3"], "ũ": ["ux", "u4"], "ụ": ["uj", "u5"], "ư": ["uw", "u7"], "ứ": ["uws", "ư1"],
        #              "ừ": ["uwf", "ư2"], "ử": ["uwr", "ư3"], "ữ": ["uwx", "ư4"],
        #              "ự": ["uwj", "ư5"], "í": ["is", "i1"], "ì": ["if", "i2"], "ỉ": ["ir", "i3"], "ị": ["ij", "i5"],
        #              "ĩ": ["ix", "i4"], "ý": ["ys", "y1"], "ỳ": ["yf", "y2"], "ỷ": ["yr", "y3"], "ỵ": ["yj", "y5"],
        #              "đ": ["dd", "d9"],
        #              "Ă": ["Aw", "A8"], "Â": ["Aa", "A6"], "Á": ["As", "A1"], "À": ["Af", "A2"], "Ả": ["Ar", "A3"],
        #              "Ã": ["Ax", "A4"], "Ạ": ["Aj", "A5"], "Ắ": ["Aws", "Ă1"], "Ổ": ["Oor", "Ô3"], "Ỗ": ["Oox", "Ô4"],
        #              "Ộ": ["Ooj", "Ô5"], "Ơ": ["Ow", "O7"],
        #              "Ằ": ["AWF", "Ă2"], "Ẳ": ["Awr", "Ă3"], "Ẵ": ["Awx", "Ă4"], "Ặ": ["Awj", "Ă5"], "Ó": ["Os", "O1"],
        #              "Ò": ["Of", "O2"], "Ỏ": ["Or", "O3"], "Õ": ["Ox", "O4"], "Ọ": ["Oj", "O5"], "Ô": ["Oo", "O6"],
        #              "Ố": ["Oos", "Ô1"], "Ồ": ["Oof", "Ô2"],
        #              "Ớ": ["Ows", "Ơ1"], "Ờ": ["Owf", "Ơ2"], "Ở": ["Owr", "Ơ3"], "Ỡ": ["Owx", "Ơ4"], "Ợ": ["Owj", "Ơ5"],
        #              "É": ["Es", "E1"], "È": ["Ef", "E2"], "Ẻ": ["Er", "E3"], "Ẽ": ["Ex", "E4"], "Ẹ": ["Ej", "E5"],
        #              "Ê": ["Ee", "E6"], "Ế": ["Ees", "Ê1"], "Ề": ["Eef", "Ê2"],
        #              "Ể": ["Eer", "Ê3"], "Ễ": ["Eex", "Ê4"], "Ệ": ["Eej", "Ê5"], "Ú": ["Us", "U1"], "Ù": ["Uf", "U2"],
        #              "Ủ": ["Ur", "U3"], "Ũ": ["Ux", "U4"], "Ụ": ["Uj", "U5"], "Ư": ["Uw", "U7"], "Ứ": ["Uws", "Ư1"],
        #              "Ừ": ["Uwf", "Ư2"], "Ử": ["Uwr", "Ư3"], "Ữ": ["Uwx", "Ư4"],
        #              "Ự": ["Uwj", "Ư5"], "Í": ["Is", "I1"], "Ì": ["If", "I2"], "Ỉ": ["Ir", "I3"], "Ị": ["Ij", "I5"],
        #              "Ĩ": ["Ix", "I4"], "Ý": ["Ys", "Y1"], "Ỳ": ["Yf", "Y2"], "Ỷ": ["Yr", "Y3"], "Ỵ": ["Yj", "Y5"],
        #              "Đ": ["Dd", "D9"]}

        self.all_word_candidates = self.get_all_word_candidates(self.word_couples)
        self.string_all_word_candidates = ' '.join(self.all_word_candidates)
        self.all_char_candidates = self.get_all_char_candidates()

    def replace_teencode(self, word):
        candidates = self.teencode_dict.get(word, None)
        if candidates is not None:
            chosen_one = 0
            if len(candidates) > 1:
                chosen_one = np.random.randint(0, len(candidates))
            return candidates[chosen_one]


    def replace_word_candidate(self, word):
        """
        Return a homophone word of the input word.
        """
        capital_flag = word[0].isupper()
        word = word.lower()
        if capital_flag and word in self.teencode_dict:
            return self.replace_teencode(word).capitalize()
        elif word in self.teencode_dict:
            return self.replace_teencode(word)

        for couple in self.word_couples:
            for i in range(2):
                if couple[i] == word:
                    if i == 0:
                        return couple[1].capitalize() if capital_flag else couple[1]
                    else:
                        return couple[0].capitalize() if capital_flag else couple[0]

    def replace_char_candidate(self, char):
        """
        return a homophone char/subword of the input char.
        """
        for couple in self.char_couples:
            for i in range(2):
                if couple[i] == char:
                    return couple[1] if i == 0 else couple[0]

    def replace_char_candidate_typo(self, char):
        """
        return a homophone char/subword of the input char.
        """
        i = np.random.randint(0, 2)

        return self.typo[char][i]

    def get_all_char_candidates(self, ):

        all_char_candidates = []
        for couple in self.char_couples:
            all_char_candidates.extend(couple)
        return all_char_candidates

    def get_all_word_candidates(self, word_couples):

        all_word_candidates = []
        for couple in self.word_couples:
            all_word_candidates.extend(couple)
        return all_word_candidates

    def remove_diacritics(self, text, onehot_label):
        """
        Replace word which has diacritics with the same word without diacritics
        Args:
            text: a list of word tokens
            onehot_label: onehot array indicate position of word that has already modify, so this
            function only choose the word that do not has onehot label == 1.
        return: a list of word tokens has one word that its diacritics was removed,
                a list of onehot label indicate the position of words that has been modified.
        """
        idx = np.random.randint(0, len(onehot_label))
        prevent_loop = 0
        while onehot_label[idx] == 1 or text[idx] == unidecode.unidecode(text[idx]) or text[idx] in string.punctuation:
            idx = np.random.randint(0, len(onehot_label))
            prevent_loop += 1
            if prevent_loop > 10:
                return False, text, onehot_label

        onehot_label[idx] = 1
        text[idx] = unidecode.unidecode(text[idx])
        return True, text, onehot_label

    def remove_letter(self, text, onehot_label):
        """
        Replace, add (or remove) a random letter in a random chosen word with a random letter
        Args:
            text: a list of word tokens
            onehot_label: onehot array indicate position of word that has already modify, so this
            function only choose the word that do not has onehot label == 1.
        return: a list of word tokens has one word that has been modified,
                a list of onehot label indicate the position of words that has been modified.
        """
        idx = np.random.randint(0, len(onehot_label))
        prevent_loop = 0
        while onehot_label[idx] == 1 or text[idx].isnumeric() or text[idx] in string.punctuation:
            idx = np.random.randint(0, len(onehot_label))
            prevent_loop += 1
            if prevent_loop > 10:
                return False, text, onehot_label

        chosen_letter = text[idx][np.random.randint(0, len(text[idx]))]
        try:
            text[idx] = re.sub(chosen_letter, '', text[idx])
        except:
            return False, text, onehot_label

        onehot_label[idx] = 1
        return True, text, onehot_label

    def replace_with_homophone_word(self, text, onehot_label):
        """
        Replace a candidate word (if exist in the word_couple) with its homophone. if successful, return True, else False
        Args:
            text: a list of word tokens
            onehot_label: onehot array indicate position of word that has already modify, so this
            function only choose the word that do not has onehot label == 1.
        return: True, text, onehot_label if successful replace, else False, text, onehot_label
        """
        candidates = [
            (i, text[i])
            for i in range(len(text))
            if text[i].lower() in self.all_word_candidates
            or text[i].lower() in self.teencode_dict.keys()
        ]
        if not candidates:
            return False, text, onehot_label

        idx = np.random.randint(0, len(candidates))
        prevent_loop = 0
        while onehot_label[candidates[idx][0]] == 1:
            idx = np.random.choice(np.arange(0, len(candidates)))
            prevent_loop += 1
            if prevent_loop > 5:
                return False, text, onehot_label

        text[candidates[idx][0]] = self.replace_word_candidate(candidates[idx][1])
        onehot_label[candidates[idx][0]] = 1
        return True, text, onehot_label

    def replace_with_homophone_letter(self, text, onehot_label):
        """
        Replace a subword/letter with its homophones
        Args:
            text: a list of word tokens
            onehot_label: onehot array indicate position of word that has already modify, so this
            function only choose the word that do not has onehot label == 1.
        return: True, text, onehot_label if successful replace, else False, None, None
        """
        candidates = []
        for i in range(len(text)):
            for char in self.all_char_candidates:
                if re.search(char, text[i]) is not None:
                    candidates.append((i, char))
                    break

        if not candidates:
            return False, text, onehot_label
        idx = np.random.randint(0, len(candidates))
        prevent_loop = 0
        while onehot_label[candidates[idx][0]] == 1:
            idx = np.random.randint(0, len(candidates))
            prevent_loop += 1
            if prevent_loop > 5:
                return False, text, onehot_label

        replaced = self.replace_char_candidate(candidates[idx][1])
        text[candidates[idx][0]] = re.sub(candidates[idx][1], replaced, text[candidates[idx][0]])

        onehot_label[candidates[idx][0]] = 1
        return True, text, onehot_label

    def replace_with_typo_letter(self, text, onehot_label):
        """
        Replace a subword/letter with its homophones
        Args:
            text: a list of word tokens
            onehot_label: onehot array indicate position of word that has already modify, so this
            function only choose the word that do not has onehot label == 1.
        return: True, text, onehot_label if successful replace, else False, None, None
        """
        # find index noise
        idx = np.random.randint(0, len(onehot_label))
        prevent_loop = 0
        while onehot_label[idx] == 1 or text[idx].isnumeric() or text[idx] in string.punctuation:
            idx = np.random.randint(0, len(onehot_label))
            prevent_loop += 1
            if prevent_loop > 10:
                return False, text, onehot_label

        index_noise = idx
        onehot_label[index_noise] = 1

        word_noise = text[index_noise]
        for j in range(0, len(word_noise)):
            char = word_noise[j]

            if char in self.typo:
                replaced = self.replace_char_candidate_typo(char)
                word_noise = word_noise[: j] + replaced + word_noise[j + 1:]
                text[index_noise] = word_noise
                return True, text, onehot_label
        return True, text, onehot_label

    def random_space_ratio(self,text, ratio = 0.1):
        """
        Randomly remove spaces from a string with a given ratio.

        Args:
          text: The string to process.
          ratio: The ratio of spaces to remove (between 0 and 1).

        Returns:
          The string with random spaces removed.
        """
        mask = np.random.rand(len(text)) < ratio

        text = [char for i, char in enumerate(text) if not (char == " " and mask[i])]

        return "".join(text)

    def add_noise(self, sentence, percent_err=0.3, num_type_err=5):
        tokens = self.tokenizer(sentence)
        onehot_label = [0] * len(tokens)

        num_wrong = int(np.ceil(percent_err * len(tokens)))
        num_wrong = np.random.randint(1, num_wrong + 1)
        if np.random.rand() < 0.05:
            num_wrong = 0

        for _ in range(0, num_wrong):
            err = np.random.randint(0, num_type_err + 1)

            if err == 0:
                _, tokens, onehot_label = self.replace_with_homophone_letter(tokens, onehot_label)
            elif err == 1 or err == 2:
                _, tokens, onehot_label = self.replace_with_typo_letter(tokens, onehot_label)
            elif err == 3:
                _, tokens, onehot_label = self.replace_with_homophone_word(tokens, onehot_label)
            elif err == 4:
                _, tokens, onehot_label = self.remove_letter(tokens, onehot_label)
            elif err == 5:
                _, tokens, onehot_label = self.remove_diacritics(tokens, onehot_label)
            else:
                continue

        tokens = ' '.join(tokens)
        tokens = self.random_space_ratio(text = tokens)

        return tokens



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
synt = SynthesizeData()
noise_train_ngrams = []
for ngram in tqdm(train_ngrams):
  noise_train_ngrams.append(synt.add_noise(ngram))

100%|██████████| 6579290/6579290 [20:51<00:00, 5255.15it/s]


In [ ]:
noise_test_data = []
for sentence in tqdm(test_data):
  noise_test_data.append(synt.add_noise(sentence))

100%|██████████| 35999/35999 [00:45<00:00, 788.56it/s]


# GHI VÀO FILE CSV

In [ ]:
import csv

with open('train_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['input','output'])
    for noise_ngram,ngram in zip(noise_train_ngrams,train_ngrams):
        writer.writerow([noise_ngram,ngram])

In [ ]:
with open('test_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['input','output'])
    for noise_sen,sen in zip(noise_test_data,test_data):
        writer.writerow([noise_sen,sen])